# FARM Building Blocks

Welcome to the FARM building blocks tutorial! There are many different ways to make use of this repository, but in this notebook, we will be going through the most import building blocks that will help you harvest the rewards of a successfully trained NLP model.

Happy FARMing!

## 1) Text Classification

GNAD (https://tblock.github.io/10kGNAD/) is a dataset of 10K German documents labelled with one of 9 classes. In this section, we are going to build a classifier for this task that is composed of Google's BERT language model and a feed forward neural network prediction head.

### Setup

In [2]:
# Let's start by adjust the working directory so that it is the root of the repository
# This should be run just once.

import os
os.chdir('../')
print("Current working directory is {}".format(os.getcwd()))

Current working directory is /Users/deepset/deepset/FARM


In [3]:
# Here are the imports we need

import torch
from farm.modeling.tokenization import BertTokenizer
from farm.data_handler.processor import GNADProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import Bert
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.experiment import initialize_optimizer
from farm.train import Trainer

Apex not installed. If you use distributed training with local rank != -1 apex must be installed.


In [4]:
# We need to fetch the right device to drive the growth of our model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices available: {}".format(device))

Devices available: cpu


### Data Handling

In [5]:
# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path="bert-base-german-cased",
    do_lower_case=False)

07/24/2019 14:37:05 - INFO - farm.modeling.tokenization_utils -   loading file https://int-deepset-models-bert.s3.eu-central-1.amazonaws.com/pytorch/bert-base-german-cased-vocab.txt from cache at /Users/deepset/.cache/torch/farm/da299cdd121a3d71e1626f2908dda0d02658f42e925a3d6abd8273ec08cf41a6.2a48e6c60dcdb582effb718237ce5894652e3b4abb94f0a4d9a857b70333308d


In [6]:
# We can test out how it will do on an example sentence

EXAMPLE_SENTENCE = "Selbst ein blindes Huhn findet mal ein Korn."
tokenizer.tokenize(EXAMPLE_SENTENCE)

['Selbst',
 'ein',
 'bl',
 '##inde',
 '##s',
 'Hu',
 '##hn',
 'findet',
 'mal',
 'ein',
 'Korn',
 '.']

In [7]:
# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor

processor = GNADProcessor(tokenizer=tokenizer,
                          max_seq_len=128,
                          data_dir="data/gnad")

In [8]:
# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

07/24/2019 14:37:14 - INFO - farm.data_handler.data_silo -   Loading train set from: data/gnad/train.csv
07/24/2019 14:37:14 - INFO - farm.data_handler.utils -   downloading and extracting file gnad to dir /Users/deepset/deepset/FARM/data
100%|██████████| 10753295/10753295 [00:28<00:00, 377680.96B/s]
07/24/2019 14:40:52 - INFO - farm.data_handler.processor -   *** Show 3 random examples ***
07/24/2019 14:40:52 - INFO - farm.data_handler.processor -   *** Example ***
ID: train-7025-0
Clear Text: 
 	label: Wissenschaft
 	text: Das Zentrum für Soziale Innovation ist beteiligt an EU-Projekt für mehr Verantwortung in der Wissenschaft. Wien - Im Verhältnis von Wissenschaft und Gesellschaft lässt sich eine Doppelbewegung ausmachen: Einerseits rücken durch die Technisierung zunehmend Artefakte der Wissenschaft in die Lebenswelt, andererseits werden Forschung und die daraus resultierenden Anwendungen immer spezialisierter und entziehen sich so zunehmend dem Verständnis der meisten. Angesichts d

07/24/2019 14:40:52 - INFO - farm.data_handler.processor -   *** Example ***
ID: train-373-0
Clear Text: 
 	label: Panorama
 	text: Innenministerium macht erneut von Durchgriffsrecht Gebrauch – Umsetzung "so rasch wie möglich". Villach – In der Hensel-Kaserne in Villach wird in den kommenden Wochen ein Containerdorf für 450 Asylwerber entstehen. Das Innenministerium macht vom Durchgriffsrecht Gebrauch, Land und Stadt sind am Freitag informiert worden, sagte Ministeriumssprecher Karl-Heinz Grundböck auf Anfrage der APA. Aufbau und Belegung sollen so rasch wie möglich vonstattengehen. Es gibt schon länger das Angebot des Verteidigungsministeriums, erläuterte Grundböck. Jetzt werde das betroffene Gelände per Verordnung zur Bundesbetreuungsstelle erklärt. Ein Verwaltungsübereinkommen mit dem Verteidigungsministerium liege bereits vor. Als nächstes werde der Bescheid erstellt. Weiters gehe es an die Planung zur Errichtung des Containerdorfs auf einem Freigelände der Kaserne. Die Villacher S

07/24/2019 14:40:52 - INFO - farm.data_handler.data_silo -   Loading dev set as a slice of train set
07/24/2019 14:40:52 - INFO - farm.data_handler.data_silo -   Took 924 samples out of train set to create dev set (dev split = 0.1)
07/24/2019 14:40:52 - INFO - farm.data_handler.data_silo -   Loading test set from: data/gnad/test.csv
07/24/2019 14:41:12 - INFO - farm.data_handler.processor -   *** Show 3 random examples ***
07/24/2019 14:41:12 - INFO - farm.data_handler.processor -   *** Example ***
ID: test-789-0
Clear Text: 
 	label: Sport
 	text: Red Bull Salzburg hat erst als drittes Team in der Geschichte das Double erfolgreich verteidigt. Die Salzburger holten sich am Mittwochabend mit einem 2:0-Sieg nach Verlängerung gegen die Wiener Austria im Klagenfurter Wörthersee-Stadion auch den Titel im ÖFB-Fußball-Cup. Zum Matchwinner für den Meister avancierten Jonatan Soriano (95.) und Felipe Pires (108.).
Tokenized: 
 	 tokens: ['Red', 'Bull', 'Salzburg', 'hat', 'erst', 'als', 'drittes

07/24/2019 14:41:12 - INFO - farm.data_handler.processor -   *** Example ***
ID: test-788-0
Clear Text: 
 	label: Wirtschaft
 	text: Zwei Ex-Mitarbeiter der Wirtschaftsprüfungs- und Steuerberatungsgesellschaft PricewaterhouseCoopers kommen wie ein Journalist vor Gericht. Luxemburg – In der Luxleaks-Affäre um Steuervorteile für internationale Konzerne in Luxemburg wird den Informanten der Prozess gemacht. Drei Männer müssen sich vom 26. April an vor einem Gericht in Luxemburg-Stadt verantworten, weil sie die Steuerdeals zwischen Unternehmen und Steuerbehörden an die Öffentlichkeit brachten. Das teilte ein Justizsprecher am Freitag in Luxemburg mit. Angeklagt sind zwei Ex-Mitarbeiter der Wirtschaftsprüfungs- und Steuerberatungsgesellschaft PricewaterhouseCoopers (PwC) sowie ein französischer Journalist. Die früheren PwC-Angestellten werden des Diebstahls, des illegalen Zugriffs auf ein Computersystem, der Weitergabe von Geschäftsgeheimnissen, des Bruchs der beruflichen Schweigepflicht un

### Modeling

In FARM, we make a strong distinction between the language model and prediction head so that you can mix and match different building blocks for your needs.

For example, in the transfer learning paradigm, you might have the one language model that you will be using for both document classification and NER. Or you perhaps you have a pretrained language model which you would like to adapt to your domain, then use for a downstream task such as question answering. 

All this is possible within FARM and requires only the replacement of a few modular components, as we shall see below.

Let's first have a look at how we might set up a model.

In [ ]:
# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "bert-base-german-cased"

language_model = Bert.load(MODEL_NAME_OR_PATH)

In [ ]:
# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 9 (the number of classes in the GNAD dataset)

LAYER_DIMS = [768, 9]

prediction_head = TextClassificationHead(layer_dims=LAYER_DIMS)

In [ ]:
# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
EMBEDS_DROPOUT_PROB = 0.1

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

### Training

In [ ]:
# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

LEARNING_RATE = 2e-5
WARMUP_PROPORTION = 0.1
N_EPOCHS = 1

optimizer, warmup_linear = initialize_optimizer(
    model=model,
    learning_rate=LEARNING_RATE,
    warmup_proportion=WARMUP_PROPORTION,
    n_examples=data_silo.n_samples("train"),
    batch_size=data_silo.batch_size,
    n_epochs=N_EPOCHS)

In [ ]:
# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 0

trainer = Trainer(
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    warmup_linear=warmup_linear,
    device=device,
)

In [ ]:
model = trainer.train(model)

# Switch to NER

In a transfer learning paradigm, there is a core computation that is shared amongst all tasks. FARM's modular structure means that you can easily swap out different building blocks to make the same language model work for many different tasks.

We can adapt the above text classification model to NER by simply switching out the processor and prediction head.

In [ ]:
# Import the new building blocks

from farm.data_handler.processor import CONLLProcessor
from farm.modeling.prediction_head import TokenClassificationHead

In [ ]:
# This processor will preprocess the data for the CoNLL03 NER task

ner_processor = CONLLProcessor(tokenizer=tokenizer,
                               max_seq_len=128,
                               data_dir="data/conll03de")

In [ ]:
# This prediction head is also a feed forward neural network but expects one
# vector per token in the input sequence and will generate a set of logits
# for each input

LAYER_DIMS = [768, 13]

ner_prediction_head = TokenClassificationHead(layer_dims=LAYER_DIMS)

In [ ]:
# We can integrate these new pieces with the rest using this code
# It is pretty much the same structure as what we had above for text classification

BATCH_SIZE = 32
EMBEDS_DROPOUT_PROB = 0.1
LEARNING_RATE = 2e-5
WARMUP_PROPORTION = 0.1
N_EPOCHS = 1
N_GPU = 0

data_silo = DataSilo(
    processor=ner_processor,
    batch_size=BATCH_SIZE)

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[ner_prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_token"],
    device=device)

optimizer, warmup_linear = initialize_optimizer(
    model=model,
    learning_rate=LEARNING_RATE,
    warmup_proportion=WARMUP_PROPORTION,
    n_examples=data_silo.n_samples("train"),
    batch_size=BATCH_SIZE,
    n_epochs=N_EPOCHS)

trainer = Trainer(
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    warmup_linear=warmup_linear,
    device=device,
)

In [ ]:
model = trainer.train(model)

# Save and load